In [ ]:
import nltk, re, string, unicodedata, inflect, glob, os
import re
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import numpy as np 
import pandas as pd                                 #for data manipulation and analysis
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
with open("/content/full_text.txt", 'rb') as nai:
  text = nai.read()

In [ ]:
import multiprocessing
import gensim 
import gensim.utils
from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [ ]:
sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(str(text))]

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
w2v = gensim.models.Word2Vec.load("models/w2v.model")

In [ ]:
!wget http://evexdb.org/pmresources/vec-space-models/PubMed-and-PMC-w2v.bin

--2020-05-31 16:15:03--  http://evexdb.org/pmresources/vec-space-models/PubMed-and-PMC-w2v.bin
Resolving evexdb.org (evexdb.org)... 130.232.253.44
Connecting to evexdb.org (evexdb.org)|130.232.253.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3317501420 (3.1G) [application/octet-stream]
Saving to: ‘PubMed-and-PMC-w2v.bin’

PubMed-and-PMC-w2v.   3%[                    ] 121.37M  1014KB/s    eta 51m 36s^C


In [ ]:
!wget http://evexdb.org/pmresources/vec-space-models/PMC-w2v.bin

--2020-05-31 16:17:18--  http://evexdb.org/pmresources/vec-space-models/PMC-w2v.bin
Resolving evexdb.org (evexdb.org)... 130.232.253.44
Connecting to evexdb.org (evexdb.org)|130.232.253.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2040351350 (1.9G) [application/octet-stream]
Saving to: ‘PMC-w2v.bin’

PMC-w2v.bin         100%[===================>]   1.90G  1.01MB/s    in 32m 41s 

2020-05-31 16:50:00 (1016 KB/s) - ‘PMC-w2v.bin’ saved [2040351350/2040351350]



In [ ]:
from gensim.test.utils import common_texts, get_tmpfile

INFO - 16:52:45: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 16:52:45: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [ ]:
path = get_tmpfile("/content/PMC-w2v.bin")

In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format ('/content/PMC-w2v.bin', binary=True)


INFO - 16:55:39: loading projection weights from /content/PMC-w2v.bin
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO - 16:56:07: loaded (2515686, 200) matrix from /content/PMC-w2v.bin


In [ ]:
print(model.most_similar(positive=['woman', 'king'], negative=['man']))


In [ ]:
model.wv.most_similar(positive=["gene"])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('genes', 0.7276425361633301),
 ('locus', 0.6418459415435791),
 ('transcript', 0.6243889331817627),
 ('promoter', 0.613724946975708),
 ('instability-correlated', 0.6043727397918701),
 ('transgene', 0.5990813970565796),
 ('gene(s)', 0.5974688529968262),
 ('operon', 0.5946967601776123),
 ('DDB_G0281803', 0.5872200727462769),
 ('tissue-specific', 0.5820614099502563)]

In [ ]:
t = time()

model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.wv.most_similar(positive=["enzyme"])


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('purified', 0.9926730990409851),
 ('subunits', 0.9760609865188599),
 ('protein', 0.9747612476348877),
 ('reductase', 0.9732197523117065),
 ('identical', 0.9704747200012207),
 ('dehydrogenase', 0.9655677676200867),
 ('subunit', 0.9628738164901733),
 ('acid', 0.9609610438346863),
 ('dioxygenase', 0.9604526162147522),
 ('alpha', 0.9596236944198608)]